In [1]:
import pandas as pd
import re

df = pd.read_csv('sms_spam.csv', names=['TÜR', 'MESAJ'])
df.drop(0,inplace=True)## df'nin ilk satırını siler.
print(df.shape) ##satır ve sütun sayısını verir.
df.head() 



(5559, 2)


,TÜR,MESAJ
1,ham,Hope you are having a good week. Just checking in
2,ham,K..give back my thanks.
3,ham,Am also doing in cbe only. But have to pay.
4,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
5,spam,okmail: Dear Dave this is your final notice to...


In [2]:
df.info() ## data setin bilgileri

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5559 entries, 1 to 5559
Data columns (total 2 columns):
TÜR      5559 non-null object
MESAJ    5559 non-null object
dtypes: object(2)
memory usage: 130.3+ KB


In [3]:
df['TÜR'].value_counts(normalize=True) ##class table frekansları (olasılık değerleri)

ham     0.865623
spam    0.134377
Name: TÜR, dtype: float64

In [4]:
data_randomized = df.sample(frac=1, random_state=44) ## sample degerlerini karıştırdık ( eğitilecek verileri seçmek için)
data_randomized.head()


,TÜR,MESAJ
797,ham,Hey company elama po mudyadhu.
4436,ham,"Watching cartoon, listening music & at eve had..."
2910,spam,Moby Pub Quiz.Win a £100 High Street prize if ...
624,ham,Meet after lunch la...
1174,ham,What happen dear. Why you silent. I am tensed


In [5]:
training_test_index = round(len(data_randomized) * 0.75) ##elimizdeki verilerin kaçının eğitileceğini hesaplıyoruz.
training_test_index 

4169

In [6]:
training_set = data_randomized[:training_test_index].reset_index(drop=True)##4169 tane eğitilecek veri
training_set.head()

,TÜR,MESAJ
0,ham,Hey company elama po mudyadhu.
1,ham,"Watching cartoon, listening music & at eve had..."
2,spam,Moby Pub Quiz.Win a £100 High Street prize if ...
3,ham,Meet after lunch la...
4,ham,What happen dear. Why you silent. I am tensed


In [7]:
test_set = data_randomized[training_test_index:].reset_index(drop=True)## geri kalan verileri test içine aldık.
test_set.head()


,TÜR,MESAJ
0,ham,No she didnt. I will search online and let you...
1,ham,"Ya ok, vikky vl c witin # mins and il reply u.."
2,ham,Hi. Wk been ok - on hols now! Yes on for a bit...
3,ham,"Hmph. Go head, big baller."
4,ham,Watching tv now. I got new job :)


In [8]:
training_set['TÜR'].value_counts(normalize=True)
## training  datasının frekans(class labellerini bulduk)

ham     0.868074
spam    0.131926
Name: TÜR, dtype: float64

In [9]:
test_set['TÜR'].value_counts(normalize=True)
## test datalasının frekans(class labellerini bulduk)

ham     0.858273
spam    0.141727
Name: TÜR, dtype: float64

In [10]:
training_set['MESAJ'] = training_set['MESAJ'].str.replace('\W', ' ')## A-Z, a-z ve 1-9 arasında olmayan degerleri siler.

training_set['MESAJ'] = training_set['MESAJ'].str.lower()## harflerin hepsini küçültür.

training_set['MESAJ'] = training_set['MESAJ'].str.split()## hepsini list yaparak ayırır.
training_set.head()

,TÜR,MESAJ
0,ham,"[hey, company, elama, po, mudyadhu]"
1,ham,"[watching, cartoon, listening, music, at, eve,..."
2,spam,"[moby, pub, quiz, win, a, 100, high, street, p..."
3,ham,"[meet, after, lunch, la]"
4,ham,"[what, happen, dear, why, you, silent, i, am, ..."


In [11]:
kelimeler = set() ## kelimeler adında bir küme oluşturduk. Tekrar eden kelimeleri engellemek için
for sms in training_set['MESAJ']:## training_set teki mesajlardaki tekrar eden kelimelerden 1 tanesini aldık.
    for word in sms:
        kelimeler.add(word)
len(kelimeler)

7432

In [12]:
sms_mesaj = {unique_word: [0] * len(training_set['MESAJ']) for unique_word in kelimeler} 
##tekrarlanmayan kelimelerle bir sözlük yapıyoruz

In [13]:
for index, sms in enumerate(training_set['MESAJ']): ##eşsiz kelimeler hangi satırda ise o satırın sütunundaki değer 1 oluyor.
    for word in sms:
        sms_mesaj[word][index] += 1
word_counts = pd.DataFrame(sms_mesaj )## sözlüğü tabloya çeviriyoruz.
word_counts.head()


,retrieve,superb,housing,chocolate,trav,rugby,3g,lyricalladie,vpod,lovingly,...,wings,visitors,be,actual,everyboy,threats,ink,08712101358,talk,queen
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)##Training_set ile sözlük tablosunu birleştirdik.
training_set_clean.head()##mesajdaki kelimeler 1 oluyor.

,TÜR,MESAJ,retrieve,superb,housing,chocolate,trav,rugby,3g,lyricalladie,...,wings,visitors,be,actual,everyboy,threats,ink,08712101358,talk,queen
0,ham,"[hey, company, elama, po, mudyadhu]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[watching, cartoon, listening, music, at, eve,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,spam,"[moby, pub, quiz, win, a, 100, high, street, p...",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,ham,"[meet, after, lunch, la]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[what, happen, dear, why, you, silent, i, am, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
spam_mesaj = training_set_clean[training_set_clean['TÜR'] == 'spam']##spam mesajlarını aldık
ham_mesaj = training_set_clean[training_set_clean['TÜR'] == 'ham']##ham mesajlarını aldık

In [16]:
p_spam = len(spam_mesaj) / len(training_set_clean)
p_ham = len(ham_mesaj) / len(training_set_clean)
print(p_spam)
print(p_ham)
##p_spam ve p_hamın olasılık değerlerini hesapladık. p(ham) , p(spam) 


0.13192612137203166
0.8680738786279684


In [17]:
n_kelimelerdeki_spam_mesaj = spam_mesaj['MESAJ'].apply(len)
n_spam = n_kelimelerdeki_spam_mesaj.sum()
print(n_spam)
n_kelimelerdeki_spam_mesaj
## spam mesajlarının bulunduğu indeksteki kelime sayılarını bulduk.

14081


2       32
25      31
27      24
34      34
45      25
47      29
48      21
49      31
54      25
67      28
71      20
79      29
87      22
101     23
115     25
126     28
134     25
137     31
141     17
150     28
151     19
154     32
164     13
166     28
171     12
222     28
229     23
240     28
243     25
250     23
        ..
3933    33
3938    31
3941    24
3951    28
3977    25
3985    14
3992    19
3999    33
4002    34
4018    31
4028    23
4030    18
4053    27
4054    29
4065    20
4066    19
4067    29
4079    30
4085    29
4091    23
4096    29
4103    23
4108    29
4110    24
4112    31
4118    29
4119    28
4121    26
4123    22
4137    23
Name: MESAJ, Length: 550, dtype: int64

In [18]:
n_kelimelerdeki_ham_mesaj = ham_mesaj['MESAJ'].apply(len)
n_ham = n_kelimelerdeki_ham_mesaj .sum()
print(n_ham)
n_kelimelerdeki_ham_mesaj 

52923


0        5
1       14
3        4
4        9
5       29
6       31
7       22
8        5
9        1
10       5
11       9
12       4
13      34
14      13
15       2
16       5
17      57
18      15
19      25
20       6
21      24
22       5
23       8
24      28
26      12
28      26
29       7
30       5
31      11
32      21
        ..
4139    13
4140    30
4141    29
4142    11
4143     6
4144    17
4145    18
4146     9
4147     7
4148     5
4149    19
4150     7
4151     9
4152    11
4153     9
4154    22
4155    25
4156    30
4157     4
4158    14
4159     8
4160    31
4161     9
4162     8
4163    13
4164     7
4165    12
4166    60
4167    17
4168    12
Name: MESAJ, Length: 3619, dtype: int64

In [19]:
parameters_spam = {unique_word:0 for unique_word in kelimeler}
parameters_ham = {unique_word:0 for unique_word in kelimeler}
parameters_ham


{'retrieve': 0,
 'superb': 0,
 'housing': 0,
 'chocolate': 0,
 'trav': 0,
 'rugby': 0,
 '3g': 0,
 'lyricalladie': 0,
 'vpod': 0,
 'lovingly': 0,
 'or2stoptxt': 0,
 'edrunk': 0,
 'answered': 0,
 'blastin': 0,
 '3lions': 0,
 'grandmas': 0,
 'hmv1': 0,
 'fires': 0,
 'professional': 0,
 'mate': 0,
 'theatre': 0,
 'bridge': 0,
 'ibm': 0,
 'delivery': 0,
 '2b': 0,
 'txttowin': 0,
 'lobby': 0,
 'model': 0,
 'takes': 0,
 '09066368327': 0,
 'kills': 0,
 'wtf': 0,
 'asked': 0,
 'eatin': 0,
 'incorrect': 0,
 'told': 0,
 'spjanuary': 0,
 'funny': 0,
 'grooved': 0,
 'ish': 0,
 'steak': 0,
 'fave': 0,
 'uncountable': 0,
 '6': 0,
 'nag': 0,
 'stop': 0,
 'g2': 0,
 'wasnt': 0,
 'veggie': 0,
 'female': 0,
 'thoughts': 0,
 'dena': 0,
 'gently': 0,
 'closes': 0,
 'exeter': 0,
 'roommates': 0,
 'urself': 0,
 '900': 0,
 'hostel': 0,
 'flat': 0,
 'reveal': 0,
 'katexxx': 0,
 'act': 0,
 'lord': 0,
 'yer': 0,
 'grave': 0,
 'finally': 0,
 'fault': 0,
 'round': 0,
 'canname': 0,
 'grab': 0,
 'tee': 0,
 'mns': 0,

In [20]:
for word in kelimeler:
    n_kelimenin_spam_sayısı= spam_mesaj[word].sum()   ##bir kelimenin spam içerisinde kaç kere olduğunu gösterir.
    p_kelime_olasılık_spam= (n_kelimenin_spam_sayısı + 1) / (n_spam + len(kelimeler))
    parameters_spam[word] = p_kelime_olasılık_spam
    
    n_kelimenin_ham_sayısı= ham_mesaj[word].sum()  
    p_kelime_olasılık_ham= (n_kelimenin_ham_sayısı + 1) / (n_ham + len(kelimeler))
    parameters_ham[word] = p_kelime_olasılık_ham
print(parameters_spam)
print(parameters_ham)
##p(d/c)= N() + 1 /Nc(classtaki kelimeler)+kelime L

{'retrieve': 0.00013945056477478735, 'superb': 4.6483521591595777e-05, 'housing': 4.6483521591595777e-05, 'chocolate': 4.6483521591595777e-05, 'trav': 0.00013945056477478735, 'rugby': 4.6483521591595777e-05, '3g': 0.0002324176079579789, 'lyricalladie': 9.296704318319155e-05, 'vpod': 9.296704318319155e-05, 'lovingly': 4.6483521591595777e-05, 'or2stoptxt': 0.00013945056477478735, 'edrunk': 4.6483521591595777e-05, 'answered': 4.6483521591595777e-05, 'blastin': 4.6483521591595777e-05, '3lions': 9.296704318319155e-05, 'grandmas': 4.6483521591595777e-05, 'hmv1': 9.296704318319155e-05, 'fires': 4.6483521591595777e-05, 'professional': 9.296704318319155e-05, 'mate': 4.6483521591595777e-05, 'theatre': 4.6483521591595777e-05, 'bridge': 4.6483521591595777e-05, 'ibm': 4.6483521591595777e-05, 'delivery': 0.0006507693022823409, '2b': 4.6483521591595777e-05, 'txttowin': 9.296704318319155e-05, 'lobby': 4.6483521591595777e-05, 'model': 4.6483521591595777e-05, 'takes': 0.0002324176079579789, '09066368327

In [21]:
def kontrol_test(mesaj):    

    
    mesaj = re.sub('\W', ' ', mesaj)
    mesaj = mesaj.lower().split()
    
    p_kelime_olasılık_spam = p_spam
    p_kelime_olasılık_ham = p_ham

    for kelime in mesaj:
        if kelime in parameters_spam:
            p_kelime_olasılık_spam *= parameters_spam[kelime]
            
        if kelime in parameters_ham:
            p_kelime_olasılık_ham *= parameters_ham[kelime]
    
    if  p_kelime_olasılık_ham >  p_kelime_olasılık_spam:
        return 'ham'
    elif  p_kelime_olasılık_spam >  p_kelime_olasılık_ham:
        return 'spam'
    else:
        return 'Veriye ihtiyaç var'
test_set['TAHMİN'] = test_set['MESAJ'].apply(kontrol_test)
test_set.head()



,TÜR,MESAJ,TAHMİN
0,ham,No she didnt. I will search online and let you...,ham
1,ham,"Ya ok, vikky vl c witin # mins and il reply u..",ham
2,ham,Hi. Wk been ok - on hols now! Yes on for a bit...,ham
3,ham,"Hmph. Go head, big baller.",ham
4,ham,Watching tv now. I got new job :),ham


In [22]:
dogrusayisi = 0
total = test_set.shape[0]
    
for row in test_set.iterrows():
    row = row[1]
    if row['TÜR'] == row['TAHMİN']:
        dogrusayisi += 1
        
print('Doğru Sayısı:', dogrusayisi)
print('Yanlış Sayısı:', total - dogrusayisi)
print('Doğruluk Oranı:', dogrusayisi/total)

Doğru Sayısı: 1372
Yanlış Sayısı: 18
Doğruluk Oranı: 0.9870503597122302


In [ ]:
def kontrol(mesaj):
  
    print(mesaj)
    
    mesaj = re.sub('\W', ' ', mesaj)
    mesaj = mesaj.lower().split()
    
    p_kelime_olasılık_spam = p_spam
    p_kelime_olasılık_ham = p_ham

    for kelime in mesaj:
        if kelime in parameters_spam:
             p_kelime_olasılık_spam*= parameters_spam[kelime]
            
        if kelime in parameters_ham:
            p_kelime_olasılık_ham*= parameters_ham[kelime]

    print('P(Spam|mesaj):', p_kelime_olasılık_spam)
    print('P(Ham|mesaj):', p_kelime_olasılık_ham)
 
    if p_kelime_olasılık_ham > p_kelime_olasılık_spam:
        print('TÜR: Ham')
    elif p_kelime_olasılık_ham < p_kelime_olasılık_spam:
        print('TÜR: Spam')
    else:
        print('EŞİTLİK')


while True:
    mesaj=input('Mesaj Giriniz:')
    
    if mesaj=='a':
        break
    else:
        kontrol(mesaj)



Mesaj Giriniz:merhaba linke tıklayınız www.google.com
merhaba linke tıklayınız www.google.com
P(Spam|mesaj): 4.187115542579867e-11
P(Ham|mesaj): 6.39634587599719e-13
TÜR: Spam
